In [375]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
from panel.interact import interact
import os
from pathlib import Path
%matplotlib inline
from dotenv import load_dotenv

In [376]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")

px.set_mapbox_access_token(map_box_api)

# Import Data

In [377]:
# Import the necessary CSVs to Pandas DataFrames
coordinates = pd.read_csv(Path("../Starter_Code/Data/neighborhoods_coordinates.csv")).drop_duplicates()
census = pd.read_csv(Path("../Starter_Code/Data/sfo_neighborhoods_census_data.csv")).drop_duplicates()

coordinates.head()
#census.head()

,Neighborhood,Lat,Lon
0,Alamo Square,37.791012,-122.402100
1,Anza Vista,37.779598,-122.443451
2,Bayview,37.734670,-122.401060
3,Bayview Heights,37.728740,-122.410980
4,Bernal Heights,37.728630,-122.443050


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [378]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    housing_units_plot = census.hvplot.bar(
        x="year",
        xlabel='Year',
        y="housing_units",
        ylabel="Housing Units",
        title="Housing Units in San Francisco from 2010 to 2016",
        width=500,
        ylim=(370000, 390000)
    ).opts(yformatter="%.0f")
    
    return housing_units_plot



def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    annual_data = census.groupby("year").mean().reset_index() #create DF for annual data
    avg_rent_plot = annual_data.hvplot.line(
        x="year",
        xlabel='Year',
        y="gross_rent",
        ylabel="Price per Square Foot",
        title="Average Gross Rent by Year",
        width=500,
    ).opts(yformatter="%.0f")
    
    return avg_rent_plot


avg_sales = census.groupby("sale_price_sqr_foot").mean().reset_index()
def average_sales_price():
    """Average Sales Price Per Year."""
    avg_ann_sales_plot = annual_data.hvplot.line(
        x="year",
        xlabel='Year',
        y="sale_price_sqr_foot",
        ylabel="Price per Square Foot",
        title="Average Sale Price by Year",
        width=500,
    ).opts(yformatter="%.0f")
    
    return avg_ann_sales_plot   


def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    neighborhood_data = census.groupby(["year","neighborhood"]).mean().reset_index() #create new DF for neighborhood data
    neighborhood_sales_plot = neighborhood_data.hvplot.line(
        x="year",
        xlabel='Year',
        y="sale_price_sqr_foot",
        ylabel="Price per Square Foot",
        title= "Average Price by Neighborhood",
        width=500,
        groupby='neighborhood'
    ).opts(yformatter="%.0f")
    
    return neighborhood_sales_plot     


def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    top_10 = census.groupby("neighborhood").mean().sort_values(by="sale_price_sqr_foot", ascending=False).head(10).reset_index()    
    top_10_plot = top_10.hvplot.bar(
        x="neighborhood",
        xlabel='Neighborhood',
        y="sale_price_sqr_foot",
        ylabel="Price per Square Foot",
        title= "Top 10 Most Expensive Neighborhoods in SFO",
        width=500,
        rot=90
    ).opts(yformatter="%.0f")
    
    return top_10_plot     



def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""        
    top_10_rent_plot = neighborhood_data.hvplot.bar(
        x='year',
        xlabel='Years',
        y=['gross_rent','sale_price_sqr_foot'],
        ylabel="Num Housing Units",
        title= "Comparison of Rent and Sales Prices of SFO Neighborhoods",
        width=500,
        rot=90,
        groupby='neighborhood'
    ).opts(yformatter="%.0f")
    
    return top_10_rent_plot 
    
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""
    return px.parallel_coordinates(
        top_10, 
        title='SFO Neighborhoods Parallel Coordinates Analysis',
        color='sale_price_sqr_foot',
        color_continuous_scale=px.colors.sequential.Aggrnyl,
        width=1500
    )

def parallel_categories():
    """Parallel Categories Plot."""
    return px.parallel_categories(
        top_10, 
        title='SFO Neighborhoods Parallel Categories Analysis',
        color='sale_price_sqr_foot',
        color_continuous_scale=px.colors.sequential.Aggrnyl,
        width=1500
    )

def neighborhood_map():
    """Neighborhood Map."""
    neighborhood_map=pd.concat([coordinates,avg_sales], axis=1, sort=True).dropna()
    neighborhood_map    
    return px.scatter_mapbox(neighborhood_map,
                             lat="Lat",
                             lon="Lon",
                             size='sale_price_sqr_foot',
                             color='gross_rent',
                             hover_name='Neighborhood',
                             title='Average Sale Price Per Square Foot and Gross Rent in San Francisco',
                             color_continuous_scale=px.colors.sequential.Aggrnyl,
                             zoom=10,
                            )


def sunburst():
    """Sunburst Plot."""
    top_2010 = neighborhood_data.loc[neighborhood_data['year']==2010].sort_values(by="sale_price_sqr_foot", ascending=False).head(10)
    top_2011 = neighborhood_data.loc[neighborhood_data['year']==2011].sort_values(by="sale_price_sqr_foot", ascending=False).head(10)
    top_2012 = neighborhood_data.loc[neighborhood_data['year']==2012].sort_values(by="sale_price_sqr_foot", ascending=False).head(10)
    top_2013 = neighborhood_data.loc[neighborhood_data['year']==2013].sort_values(by="sale_price_sqr_foot", ascending=False).head(10)
    top_2014 = neighborhood_data.loc[neighborhood_data['year']==2014].sort_values(by="sale_price_sqr_foot", ascending=False).head(10)
    top_2015 = neighborhood_data.loc[neighborhood_data['year']==2015].sort_values(by="sale_price_sqr_foot", ascending=False).head(10)
    top_2016 = neighborhood_data.loc[neighborhood_data['year']==2016].sort_values(by="sale_price_sqr_foot", ascending=False).head(10)

    sunburst_data = pd.concat([top_2010,top_2011,top_2012,top_2013,top_2014,top_2015,top_2016], axis=0, sort=True).dropna().reset_index()
    
    burst_plot = px.sunburst(
        sunburst_data, 
        path = ['year', 'neighborhood'],
        values='sale_price_sqr_foot', 
        color='gross_rent',
        color_continuous_scale=px.colors.sequential.Blues,
        title='Cost Analysts of Most Expensive neighborhoods in San Francisco by Year',
        width=1000,
        height=1000,
    )
    
    return burst_plot


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [379]:
# Create a Title for the Dashboard
title = '#San Francisco Housing Dashboard'

# Create a tab layout for the dashboard
cover_page = pn.Column(
    title
)

market_overview = pn.Column(
    "## SFO Housing Market Overview", 
    housing_units_per_year(), 
    average_gross_rent(), 
    average_sales_price()
)

neighborhood_analysis = pn.Column(
    "## Neighborhood Analysis",
    average_price_by_neighborhood(),
    top_most_expensive_neighborhoods(),
    most_expensive_neighborhoods_rent_sales()
)

parallel_plots = pn.Column( 
    "## Neighborhood Analysis cont.",
    parallel_categories(),
    parallel_coordinates()
)

interactive_map = pn.Column( 
    "## Neighborhood Map",
    neighborhood_map()
)

burst = pn.Column( 
    "## Summary of most expensive neighborhoods by year",
    sunburst()
)


SFO_housing_dashboard = pn.Tabs(
    ("Cover Page",cover_page),
    ("Introduction",interactive_map),
    ("Market Overview", market_overview), 
    ("Neighborhood Analysis", neighborhood_analysis),
    ("Parallel Plots", parallel_plots),
    ("Summary", burst),
)

# Create the dashboard

## Serve the Panel Dashboard

In [380]:
# Serve the# dashboard
SFO_housing_dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
    [1] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
    [2] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] HoloViews(Curve)
        [3] HoloViews(Curve)
    [3] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [2] HoloViews(Bars)
        [3] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
    [4] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [5] Column
        [0] Markdown(str)
        [1] Plotly(Figure)

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [338]:
#housing_units_per_year()

In [339]:
#average_gross_rent()

In [340]:
#average_sales_price()

In [341]:
#average_price_by_neighborhood()

In [342]:
#top_most_expensive_neighborhoods()

In [361]:
#most_expensive_neighborhoods_rent_sales()

In [343]:
#neighborhood_map().show()

In [344]:
#parallel_categories()

In [345]:
#parallel_coordinates()

In [346]:
#sunburst()